In [1]:
import numpy as np
import pandas as pd
import pydicom
%matplotlib inline
import matplotlib.pyplot as plt
import keras  
import json
from skimage.transform import resize
# from skimage import data, color
from skimage.transform import rescale, resize, downscale_local_mean    
from keras.models import model_from_json
# from tensorflow.keras.applications.vgg16 import preprocess_input
from IPython.display import Image

Using TensorFlow backend.


In [2]:
# %store -r best_threshold                 #this works only if I rerun build and train notebook right before running this
# print(best_threshold)

In [2]:
# This function reads in a .dcm file, checks the important fields for our device, and returns a numpy array
# of just the imaging data

def check_dicom(filename): 
    # todo
    
    print('Load file {} ...'.format(filename))
    ds = pydicom.dcmread(filename)       
    img = ds.pixel_array
      
    if (ds.BodyPartExamined != "CHEST"):
        print ("Body part not chest, unable to process this file")
        return
    
    if ds.Modality != 'DX' :
        print("Modality not DX(X-Ray), unable to process this file")
        return
    
    if ds.PatientPosition not in ['PA', 'AP']:        
        print("Patient position not PA or AP, unable to process this file")
        return
    
    else:    
        print("File processed correctly")    
        return img
    
    
    
# This function takes the numpy array output by check_dicom and 
# runs the appropriate pre-processing needed for our model input

def preprocess_image(img, img_mean,img_std, img_size):    
    # todo
    img = (img - img_mean)/img_std
    img = resize(img, img_size)
    
    return img

# This function loads in our trained model w/ weights and compiles it 

def load_model(model_path, weight_path):
    # todo
    json_file = open(model_path, 'r')
    model_path = json_file.read()
    json_file.close()
    model = model_from_json(model_path)
    #model.summary()
    
    model.load_weights(weight_path)
    
    return model

# This function uses our device's threshold parameters to predict whether or not
# the image shows the presence of pneumonia using our trained model

def predict_image(model, img, thresh): 
    # todo    
    pred_Y = model.predict(img, batch_size = 1, verbose = True)  
    prediction=0
    print(pred_Y)
    if pred_Y > thresh:
        prediction=1
    return prediction 

In [3]:
test_dicoms = ['test1.dcm','test2.dcm','test3.dcm','test4.dcm','test5.dcm','test6.dcm']

model_path = "my_model.json"
weight_path="{}_my_model.best.hdf5".format('xray_class')

IMG_SIZE=(1, 224,224, 3) # This might be different if you did not use vgg16
 
img_mean = 4.667375 # loads the mean image value they used during training preprocessing
img_std = 67.51837 # loads the std dev image value they used during training preprocessing

my_model = load_model(model_path, weight_path) #loads model
thresh = 0.53166294           #best_threshold   #loads the threshold they chose for model classification 

# use the .dcm files to test your prediction
for i in test_dicoms:
    
    img = np.array([])
    img = check_dicom(i)
    
    
    if img is None:
        continue
    
#     img_mean = np.mean(img)
#     img_std = np.std(img)       
    img_proc = preprocess_image(img, img_mean,img_std, IMG_SIZE)
    print(img_proc.shape)
    pred = predict_image(my_model,img_proc,thresh)
    print(pred)

Load file test1.dcm ...
File processed correctly
(1, 224, 224, 3)
1/1 [==============================] - 2s 2s/step
[[0.5034862]]
0
Load file test2.dcm ...
File processed correctly
(1, 224, 224, 3)
1/1 [==============================] - 1s 1s/step
[[0.49512303]]
0
Load file test3.dcm ...
File processed correctly
(1, 224, 224, 3)
1/1 [==============================] - 1s 1s/step
[[0.5023626]]
0
Load file test4.dcm ...
Body part not chest, unable to process this file
Load file test5.dcm ...
Modality not DX(X-Ray), unable to process this file
Load file test6.dcm ...
Patient position not PA or AP, unable to process this file
